In [209]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os
%matplotlib inline

from binance.um_futures import UMFutures
from binance.spot import Spot
from binance.lib.utils import config_logging
from binance.error import ClientError

import requests
requests.packages.urllib3.util.ssl_.DEFAULT_CIPHERS += 'HIGH:!DH:!aNULL'
import logging
logging.getLogger().setLevel(logging.WARNING)
from time import sleep
import socket
socket.getaddrinfo('localhost', 8080)

import warnings
warnings.filterwarnings('ignore')

***
## **Collecting Market Data**

In [3]:
spot_base_url='https://api.binance.com'
future_base_url = 'https://fapi.binance.com'

try:
    os.mkdir('data')
except OSError:
    pass
    
columns = ['open time', 'open', 'high', 'low', 'close', 'volume', 'close time', 'quote asset volume ', 'number of trades', 'taker buy asset volume', 'taker buy quote asset volume', 'ignore']

In [31]:
def get_spot_historical_kline(symbol = "BTCUSDT", interval = "1m", start_time=[2019, 9, 8, 0, 0, 0], end_time=[2023, 7, 19, 23, 59, 59], save=True):
    if end_time is None:
        end = int(datetime.now().timestamp()*1000)
    else:
        end = int(datetime(*end_time).timestamp()*1000)
    end_time = datetime.fromtimestamp(end/1000).strftime('%Y-%m-%d')
    start = int(datetime(*start_time).timestamp()*1000)
    start_time = datetime.fromtimestamp(start/1000).strftime('%Y-%m-%d')
    
    data = pd.DataFrame()
    last_data = pd.DataFrame()
    collecting = start
    file_list = os.listdir("data/")
    for filename in file_list:
        split = filename.split("_")
        if split[0] == symbol and split[1] == "spot" and split[2]==start_time and split[4]==interval+".csv":
            try:
                print("Already existing data found. Reloading...")
                last_data = pd.read_csv('data/'+filename)
                last_data['open time'] = pd.to_datetime(last_data['open time'])
                last_data['close time'] = pd.to_datetime(last_data['close time'])
                collecting = min(int(last_data.iloc[-1,6].timestamp()*1000+1), end)
                print("Data reloaded. Continue to collect...")
                break
            except IOError as e:
                break
    
    url = spot_base_url+"/api/v3/klines?symbol="+symbol+"&interval="+interval+"&limit=1000"+"&endTime="+str(end)
    
    request_attempt=0
    while True:
        request_attempt+=1
        response = requests.get(url+"&startTime="+str(collecting), verify=False)
        
        if response.status_code == 200:
            request_attempt=0
            batch_data = pd.DataFrame(response.json())
            
            if batch_data.empty:
                break
            
            data = pd.concat([data, batch_data], ignore_index=True)
            
            collecting = min(batch_data.iloc[-1,6]+1, end)
            
            print("progress:", round((collecting-start)/(end-start)*100,3), "%", end="\r")
            
            if collecting==end:
                break

        else:
            print('Request failed with status code:', response.status_code, ", request attempt:", request_attempt, "/3")
            if request_attempt>=3:
                break
            continue
                
    if len(data.columns)>6:
        data[0] = pd.to_datetime(data[0], unit='ms')
        data[6] = pd.to_datetime(data[6], unit='ms')
        data.columns = columns
    data = pd.concat([last_data, data], ignore_index=True)
    
    end_time = datetime.fromtimestamp(collecting/1000).strftime('%Y-%m-%d')
    
    print("progress: 100.00%")
    if save:
        data.to_csv('data/'+symbol+'_spot_'+start_time+'_'+end_time+'_'+interval+'.csv', index=False)
    
    return data

In [59]:
def get_future_historical_kline(symbol = "BTCUSDT", interval = "1m", contract_type = "PERPETUAL", start_time=[2019, 9, 8, 0, 0, 0], end_time=[2023, 7, 19, 23, 59, 59], save=True):
    if end_time is None:
        end = int(datetime.now().timestamp()*1000)
    else:
        end = int(datetime(*end_time).timestamp()*1000)
    end_time = datetime.fromtimestamp(end/1000).strftime('%Y-%m-%d')
    start = int(datetime(*start_time).timestamp()*1000)
    start_time = datetime.fromtimestamp(start/1000).strftime('%Y-%m-%d')
    
    data = pd.DataFrame()
    last_data = pd.DataFrame()
    collecting = start
    
    file_list = os.listdir("data/")    
    for filename in file_list:
        split = filename.split("_")
        if split[0] == symbol and split[1:-3] == contract_type.split("_") and split[-3]==start_time and split[-1]==interval+".csv":
            try:
                print("Already existing data found. Reloading...")
                last_data = pd.read_csv('data/'+filename)
                last_data['open time'] = pd.to_datetime(last_data['open time'])
                last_data['close time'] = pd.to_datetime(last_data['close time'])
                collecting = min(int(last_data.iloc[-1,6].timestamp()*1000+1), end)
                print("Data reloaded. Continue to collect...")
                break
            except IOError as e:
                break
    
    url = future_base_url+"/fapi/v1/continuousKlines?pair="+symbol+"&contractType="+contract_type+"&interval="+interval+"&limit=1500"+"&endTime="+str(end)
    
    request_attempt=0
    while True:
        request_attempt+=1
        response = requests.get(url+"&startTime="+str(collecting), verify=False)
        
        if response.status_code == 200:
            request_attempt=0
            batch_data = pd.DataFrame(response.json())
            
            if batch_data.empty:
                break
            
            data = pd.concat([data, batch_data], ignore_index=True)
            
            collecting = min(batch_data.iloc[-1,6]+1, end)
            
            print("progress:", round((collecting-start)/(end-start)*100,3), "%", end="\r")
            
            if collecting==end:
                break

        else:
            print('Request failed with status code:', response.status_code, ", request attempt:", request_attempt, "/3")
            if request_attempt>=3:
                break
            continue
        
    if len(data.columns)>6:
        data[0] = pd.to_datetime(data[0], unit='ms')
        data[6] = pd.to_datetime(data[6], unit='ms')
        data.columns = columns
    data = pd.concat([last_data, data], ignore_index=True)
    
    end_time = datetime.fromtimestamp(collecting/1000).strftime('%Y-%m-%d')
    
    print("progress: 100.00%")
    if save:
        data.to_csv('data/'+symbol+'_'+contract_type+'_'+start_time+'_'+end_time+'_'+interval+'.csv', index=False)
        
    return data

In [6]:
def get_future_historical_funding_rate(symbol = "BTCUSDT", start_time=[2019, 9, 8, 23, 59, 59], end_time=[2023, 7, 14, 23, 59, 59], save=True):
    if end_time is None:
        end = int(datetime.now().timestamp()*1000)
    else:
        end = int(datetime(*end_time).timestamp()*1000)
    end_time = datetime.fromtimestamp(end/1000).strftime('%Y-%m-%d')
    start = int(datetime(*start_time).timestamp()*1000)
    start_time = datetime.fromtimestamp(start/1000).strftime('%Y-%m-%d')
    
    data = pd.DataFrame()
    last_data = pd.DataFrame()
    collecting = start
    
    file_list = os.listdir("data/")    
    for filename in file_list:
        split = filename.split("_")
        if split[0] == symbol and split[1] == "funding rate" and split[2]==start_time:
            try:
                print("Already existing data found. Reloading...")
                last_data = pd.read_csv('data/'+filename)
                last_data['fundingTime'] = pd.to_datetime(last_data['fundingTime'])
                collecting = min(int(last_data.iloc[-1, 1].timestamp()*1000+1), end)
                print("Data reloaded. Continue to collect...")
                break
            except IOError as e:
                break
    
    url = future_base_url+"/fapi/v1/fundingRate?symbol="+symbol+"&limit=1000"+"&endTime="+str(end)
    
    while True:
        response = requests.get(url+"&startTime="+str(collecting), verify=False)
        
        if response.status_code == 200:
            batch_data = pd.DataFrame(response.json())
            
            if batch_data.empty:
                break
            
            data = pd.concat([data, batch_data], ignore_index=True)
            
            collecting = min(batch_data.iloc[-1,1]+1, end)
            
            print("progress:", round((collecting-start)/(end-start)*100,3), "%", end="\r")
            
            if collecting==end:
                break

        else:
            print('Request failed with status code:', response.status_code)
            break
    
    if "fundingTime" in data.columns:
        data['fundingTime'] = pd.to_datetime(data['fundingTime'], unit='ms')
    data = pd.concat([last_data, data], ignore_index=True)
    
    print("progress: 100.00%")
    if save:
        data.to_csv('data/'+symbol+'_funding rate_'+start_time+'_'+end_time+'.csv', index=False)

    return data


In [7]:
def check_omission(data, interval, time_col_name):
    time_interval = pd.Timedelta(interval)
    if interval=='1M':
        time_interval = pd.Timedelta(days=30)
    omission_check = data[time_col_name].diff()
    mask = omission_check > interval
    
    omission = pd.DataFrame(data.loc[mask,time_col_name])
    
    omission['omission'] = omission_check[mask].apply(lambda x:x.total_seconds()/time_interval.total_seconds()-1)
    omission.set_index(time_col_name, inplace=True)
    
    if not omission.empty:
        mask = omission['omission']>0.05
        omission = omission[mask]
        
    if omission.empty:
        print('no omission')
        return 
    
    print(omission)

### How to use

*get_spot_historical_kline() & get_future_historical_kline()*
- params
    - symbol : ex) btcusdt
    - interval : 1s, 1m, 3m, 5m, 30m, 1h, 6h, 8h, 12h, 1d, 3d, 5d, 1M, ...
    - start time, end time : [year, month, date, hour, minute, second] format

        Note that data is available from start_time=[2019, 9, 8, 23, 59, 59] to end_time=[2023, 7, 15, 23, 59, 59].
    - save : T/F 
    - contract_type : PERPETUAL, CURRENT_QUARTER, NEXT_QUARTER

*get_future_hostorical_funding_rate()*
- params
    - symbol
    - start time, end time
    - save

*check_omission()*
- params
    - data : collected DataFrame
    - interval : the correct value of the time interval between consecutive rows should be
    - time_col_name : time column name in the DataFrame

Here are some example usages of the functions.

In [408]:
# Get the BTCUSDT spot price at 1-minute intervals from January 1, 2023, to July 1, 2023.
btcusdt_spot=get_spot_historical_kline(symbol="BTCUSDT",  interval="1m", start_time=[2023, 1, 1, 0, 0, 0], end_time=[2023, 7, 1, 23, 59, 59], save=True)
btcusdt_spot

progress: 100.00%%


,open time,open,high,low,close,volume,close time,quote asset volume,number of trades,taker buy asset volume,taker buy quote asset volume,ignore
0,2022-12-31 15:00:00,16586.43000000,16594.45000000,16585.86000000,16594.40000000,129.82164000,2022-12-31 15:00:59.999,2153706.29540410,2828,81.39593000,1350344.10410600,0
1,2022-12-31 15:01:00,16594.40000000,16596.54000000,16592.45000000,16594.90000000,107.12893000,2022-12-31 15:01:59.999,1777741.42126950,2710,54.05364000,896999.97754410,0
2,2022-12-31 15:02:00,16594.90000000,16595.50000000,16593.06000000,16594.65000000,68.18266000,2022-12-31 15:02:59.999,1131447.57227810,1736,39.60651000,657255.26228630,0
3,2022-12-31 15:03:00,16594.30000000,16599.44000000,16594.30000000,16597.26000000,124.52513000,2022-12-31 15:03:59.999,2066709.87139770,2420,76.73935000,1273615.50289060,0
4,2022-12-31 15:04:00,16596.77000000,16601.04000000,16596.77000000,16599.50000000,137.54759000,2022-12-31 15:04:59.999,2283200.05317030,2658,65.37396000,1085170.49582850,0
...,...,...,...,...,...,...,...,...,...,...,...,...
261995,2023-07-01 14:55:00,30610.80000000,30617.45000000,30599.93000000,30599.93000000,40.80212000,2023-07-01 14:55:59.999,1249115.68018490,769,17.96345000,549957.76064320,0
261996,2023-07-01 14:56:00,30599.93000000,30602.90000000,30599.93000000,30602.90000000,20.05013000,2023-07-01 14:56:59.999,613583.36489810,409,7.27837000,222734.73150050,0
261997,2023-07-01 14:57:00,30602.89000000,30602.90000000,30602.89000000,30602.89000000,2.45220000,2023-07-01 14:57:59.999,75044.41223740,247,0.53794000,16462.52402600,0
261998,2023-07-01 14:58:00,30602.90000000,30613.83000000,30602.89000000,30613.83000000,13.38363000,2023-07-01 14:58:59.999,409632.15332210,460,10.14656000,310544.62872040,0


In [410]:
# Check whether there was any omission.
check_omission(btcusdt_spot, '1m', 'open time')

                     omission
open time                    
2023-03-24 14:00:00      80.0


Note that trading was temporarily ceased in Binance on March 24, 2023.

In [382]:
# Get all available BTCUSDT perpetual prices at 1-minute intervals.
btcusdt_perp = get_future_historical_kline(symbol="BTCUSDT",  interval='1d', start_time = [2019, 9, 8, 23, 59, 59], end_time=[2023, 7, 14, 23, 59, 59], save=True)
btcusdt_perp

progress: 100.00%


,open time,open,high,low,close,volume,close time,quote asset volume,number of trades,taker buy asset volume,taker buy quote asset volume,ignore
0,2022-07-02,19272.00,19456.00,18959.00,19240.90,382851.525,2022-07-02 23:59:59.999,7361597905.72379,2976646,193273.081,3716917943.26660,0
1,2022-07-03,19241.00,19645.40,18780.10,19305.90,449508.331,2022-07-03 23:59:59.999,8622450486.10630,3298282,228761.251,4389214837.80957,0
2,2022-07-04,19306.00,20350.00,19046.30,20226.60,633310.187,2022-07-04 23:59:59.999,12445691915.01023,4682514,329571.738,6479246965.72408,0
3,2022-07-05,20226.30,20749.20,19291.60,20170.00,854238.867,2022-07-05 23:59:59.999,17057063374.95285,6376734,433546.901,8659958016.15215,0
4,2022-07-06,20170.10,20667.60,19753.00,20554.10,682076.458,2022-07-06 23:59:59.999,13764942073.24180,5228369,350174.613,7068962780.45838,0
...,...,...,...,...,...,...,...,...,...,...,...,...
360,2023-06-27,30260.10,30998.50,30213.00,30683.20,406073.348,2023-06-27 23:59:59.999,12421440922.16078,4326474,203324.100,6219800032.79922,0
361,2023-06-28,30683.20,30701.80,29804.60,30066.30,416656.434,2023-06-28 23:59:59.999,12598592811.65844,4000324,201233.626,6085170656.40759,0
362,2023-06-29,30066.30,30837.00,30036.10,30439.90,382072.846,2023-06-29 23:59:59.999,11648327668.80449,3628731,193794.917,5908416358.03994,0
363,2023-06-30,30439.90,31300.00,29500.00,30460.20,809331.305,2023-06-30 23:59:59.999,24662962533.50455,6203315,399225.344,12168565748.27174,0


In [384]:
# check omission
check_omission(btcusdt_perp, '1d', 'open time')

no omission


In [360]:
# Get the funding rate of BTCUSDT
btcusdt_funding = get_future_historical_funding_rate(symbol = "BTCUSDT", start_time=[2019, 9, 8, 23, 59, 59], end_time=[2023, 7, 15, 23, 59, 59], save=True)
btcusdt_funding

progress: 100.000%


,symbol,fundingTime,fundingRate
0,BTCUSDT,2019-09-10 08:00:00,0.00010000
1,BTCUSDT,2019-09-10 16:00:00,0.00010000
2,BTCUSDT,2019-09-11 00:00:00,0.00010000
3,BTCUSDT,2019-09-11 08:00:00,0.00010000
4,BTCUSDT,2019-09-11 16:00:00,0.00010000
...,...,...,...
4208,BTCUSDT,2023-07-14 00:00:00,0.00010000
4209,BTCUSDT,2023-07-14 08:00:00,0.00010000
4210,BTCUSDT,2023-07-14 16:00:00,0.00010000
4211,BTCUSDT,2023-07-15 00:00:00,-0.00007313


In [364]:
# Check omission
check_omission(btcusdt_funding, '8h', 'fundingTime')

no omission


**Do it by yourself**

modify the parameters below as you intend to collect data.

In [228]:
start_time=[2019, 9, 8, 23, 59, 59]
end_time=[2023, 7, 30, 23, 59, 59]
pool = ['ETH']
# pool = ['BTC', 'ETH', 'DOGE', 'BNB', 'XRP', 'LTC', 'EOS', 'LINK', 'ADA', 'DOT', 'UNI', 'THETA', 'XLM', 'EGLD', 'TRX', 'BCH', 'ETC', 'FTL']
# pool = ['LTC', 'EOS', 'LINK', 'DOT', 'UNI', 'THETA', 'XLM', 'EGLD', 'TRX', 'BCH', 'ETC', 'FTL']
interval = "1m"
to_collect = {'spot':False, 'future_perpetual':False, 'future_current_quarter':True, 'future_next_quarter':True,'fundingRate':False}

Then, run the code below to start collecting data. 

In [229]:
stop=False 
for sym in pool:
    symbol = sym+'USDT'
    
    left = to_collect.copy()
    while not stop and (left['spot'] or left['future_perpetual'] or left['future_current_quarter'] or left['future_next_quarter'] or left['fundingRate']):
        try:
            if left['spot']:
                print(sym+" spot data collecting...")
                spot_data = get_spot_historical_kline(symbol, interval = interval, start_time = start_time, end_time = end_time, save=True)
                #check_omission(spot_data, interval, 'close time')
                left['spot'] = False
            if left['future_perpetual']:
                print(sym+" future perpetual data collecting...")
                future_perpetual_data = get_future_historical_kline(symbol, contract_type='PERPETUAL', interval = interval, start_time = start_time, end_time = end_time, save=True)
                #check_omission(future_perpetual_data, interval, 'close time')
                left['future_perpetual'] = False
            if left['future_current_quarter']:
                print(sym+" future_current_quarter data collecting...")
                future_current_data = get_future_historical_kline(symbol, contract_type='CURRENT_QUARTER', interval = interval, start_time = start_time, end_time = end_time, save=True)
                #check_omission(future_current_data, interval, 'close time')
                left['future_current_quarter'] = False
            if left['future_next_quarter']:
                print(sym+" future_next_quarter data collecting...")
                future_next_data = get_future_historical_kline(symbol, contract_type='NEXT_QUARTER', interval = interval, start_time = start_time, end_time = end_time, save=True)
                #check_omission(future_next_data, interval, 'close time')
                left['future_next_quarter'] = False
            if left['fundingRate']:
                print(sym+" funding rate data collecting...")
                funding_rate_data = get_future_historical_funding_rate(symbol, end_time=None, save=True)
                #check_omission(funding_rate_data, '8h', 'fundingTime')
                left['fundingRate'] = False
        except KeyboardInterrupt:
            stop=True
            break
        except Exception:
            continue

ETH future_current_quarter data collecting...
Already existing data found. Reloading...
Data reloaded. Continue to collect...
progress: 100.00%%
ETH future_next_quarter data collecting...
Already existing data found. Reloading...
Data reloaded. Continue to collect...
progress: 100.00%%


In [23]:
df = get_future_historical_kline(symbol = "BTCUSDT", interval = "1d", contract_type = "NEXT_QUARTER", start_time=[2019, 9, 8, 0, 0, 0], end_time=[2023, 7, 19, 23, 59, 59], save=True)
df[df['number of trades']>0]

progress: 100.00%


,open time,open,high,low,close,volume,close time,quote asset volume,number of trades,taker buy asset volume,taker buy quote asset volume,ignore
0,2021-03-16,59114.2,60698.5,58420.1,60698.5,298.538,2021-03-16 23:59:59.999,17749078.7655,3334,145.366,8647387.4587,0
1,2021-03-17,60789.5,63130.5,57671.3,63086.2,474.918,2021-03-17 23:59:59.999,28473443.4978,5812,242.007,14487108.2516,0
2,2021-03-18,63129.5,64618.1,61129.5,61569.0,521.438,2021-03-18 23:59:59.999,32673653.0841,6306,265.480,16633748.0978,0
3,2021-03-19,61549.2,63857.3,59947.4,62236.2,504.428,2021-03-19 23:59:59.999,31483309.8634,6226,253.847,15845598.0040,0
4,2021-03-20,62150.0,64838.1,61922.0,62511.9,504.023,2021-03-20 23:59:59.999,31908627.1644,6284,251.445,15920848.4611,0
...,...,...,...,...,...,...,...,...,...,...,...,...
543,2023-06-26,30881.2,31080.0,30340.0,30688.1,450.728,2023-06-26 23:59:59.999,13833734.1935,7693,231.688,7110756.3810,0
544,2023-06-27,30682.4,31492.2,30640.1,31201.8,435.758,2023-06-27 23:59:59.999,13513676.4571,7596,240.315,7450574.4598,0
545,2023-06-28,31196.2,31211.5,30313.9,30574.8,557.232,2023-06-28 23:59:59.999,17159892.6736,11375,291.071,8964746.8165,0
546,2023-06-29,30573.9,31358.2,30522.3,30998.1,531.893,2023-06-29 23:59:59.999,16461078.8033,10996,231.167,7159130.3384,0


Note that you may use each individual function to collect specific data if you don't need all three datasets.